# Сравнение поведения пользователей музыкального сервиса

На данных Яндекс.Музыки мы сравним поведение пользователей двух городов - Москвы и Санкт-Петербурга.

**Цель исследования** — проверить три гипотезы:
1. Активность пользователей зависит от дня недели. Причём в Москве и Петербурге это проявляется по-разному.
2. В понедельник утром в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города. 
3. Москва и Петербург предпочитают разные жанры музыки. В Москве чаще слушают поп-музыку, в Петербурге — русский рэп.

**Ход исследования**

О качестве данных ничего не известно. Поэтому перед проверкой гипотез понадобится обзор данных. 

Мы проверим данные на ошибки и оценим их влияние на исследование. Затем, на этапе предобработки мы поищем возможность исправить самые критичные ошибки данных.
 
Таким образом, исследование пройдёт в *три этапа*:
 1. Обзор данных.
 2. Предобработка данных.
 3. Проверка гипотез.

## Обзор данных

In [1]:
# импорт библиотеки pandas
import pandas as pd

In [2]:
# чтение файла с данными и сохранение в df
df = pd.read_csv('/datasets/yandex_music_project.csv') 

In [3]:
# получение первых 5 строк таблицы df
df.head() 

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday


In [4]:
# получение общей информации о данных в таблице df
df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63848 non-null  object
 2   artist    57876 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


Итак, в таблице семь столбцов. Тип данных во всех столбцах — `object`.

Согласно документации к данным:
* `userID` — идентификатор пользователя;
* `Track` — название трека;  
* `artist` — имя исполнителя;
* `genre` — название жанра;
* `City` — город пользователя;
* `time` — время начала прослушивания;
* `Day` — день недели.

В названиях колонок видны *три нарушения стиля*:
1. Строчные буквы сочетаются с прописными.
2. Встречаются пробелы.
3. Два слова написаны в стиле "CamelCase".

Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.

**Выводы**

В каждой строке таблицы — данные о прослушанном треке. Часть колонок описывает саму композицию: название, исполнителя и жанр. Остальные данные рассказывают о пользователе: из какого он города, когда он слушал музыку. 

Предварительно можно утверждать, что, данных достаточно для проверки гипотез. Но встречаются пропуски в данных, а в названиях колонок — расхождения с хорошим стилем.

Чтобы двигаться дальше, нужно устранить проблемы в данных.

## Предобработка данных
Для начала исправим стиль в заголовках столбцов, исключим пропуски. Затем проверим данные на дубликаты.

### Стиль заголовков
Выведем на экран названия столбцов:

In [5]:
df.columns # перечень названий столбцов таблицы df

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')

Приведём названия в соответствие с хорошим стилем:
* несколько слов в названии запишем в «змеином_регистре»,
* все символы сделаем строчными,
* устраним пробелы.

Для этого переименуем колонки так:
* `'  userID'` → `'user_id'`;
* `'Track'` → `'track'`;
* `'  City  '` → `'city'`;
* `'Day'` → `'day'`.

In [6]:
df = df.rename(columns={'  userID': 'user_id', 'Track': 'track', '  City  ': 'city', 'Day': 'day'}, )# переименование столбцов

Проверим результат. Для этого ещё раз выведем на экран названия столбцов:

In [7]:
df.columns # проверка результатов - перечень названий столбцов

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')

Как мы видим, все названия теперь соответствуют хорошему стилю, можем продолжать предобработку.

### Пропуски значений
Сначала посчитаем, сколько в таблице пропущенных значений. Для этого достаточно двух методов `pandas`:

In [8]:
df.isna().sum() # подсчёт пропусков

user_id       0
track      1231
artist     7203
genre      1198
city          0
time          0
day           0
dtype: int64

Не все пропущенные значения влияют на исследование. Так в `track` и `artist` пропуски не важны для yашей работы. Достаточно заменить их явными обозначениями.

Но пропуски в `genre` могут помешать сравнению музыкальных вкусов в Москве и Санкт-Петербурге. На практике было бы правильно установить причину пропусков и восстановить данные. Такой возможности нет в учебном проекте. Придётся:
* заполнить и эти пропуски явными обозначениями,
* оценить, насколько они повредят расчётам. 

Заменим пропущенные значения в столбцах `track`, `artist` и `genre` на строку `'unknown'`. Для этого создадим список `columns_to_replace`, переберём его элементы циклом `for` и для каждого столбца выполним замену пропущенных значений:

In [9]:
columns_to_replace = ['track', 'artist', 'genre'] 
for column in columns_to_replace: # перебор названий столбцов в цикле и замена пропущенных значений на 'unknown'
    df[column] = df[column].fillna('unknown')

Убедитесь, что в таблице не осталось пропусков. Для этого ещё раз посчитайте пропущенные значения.

In [10]:
df.isna().sum() # контрольный подсчёт пропусков

user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64

В наших данных больше не осталось пропущенных значений.

### Дубликаты
В завершение предобработки данных мы избавимя от дубликатов. Начнём с явных дубликатов в таблице. Для этого понадобится всего одна команда:

In [11]:
df.duplicated().sum() # подсчёт явных дубликатов

3826

Теперь вызовем специальный метод `pandas`, чтобы удалить явные дубликаты:

In [12]:
df = df.drop_duplicates().reset_index(drop=True) # удаление явных дубликатов (с удалением старых индексов и формированием новых)

Ещё раз посчитаем явные дубликаты в таблице — убедимся, что полностью от них избавились:

In [13]:
df.duplicated().sum() # проверка на отсутствие дубликатов

0

Теперь избавимся от неявных дубликатов в колонке `genre`. Например, название одного и того же жанра может быть записано немного по-разному. Такие ошибки тоже повлияют на результат исследования.

Выведем на экран список уникальных названий жанров, отсортированный в алфавитном порядке. *Для этого*:
* извлечём нужный столбец датафрейма, 
* применим к нему метод сортировки,
* для отсортированного столбца вызовем метод, который вернёт уникальные значения из столбца.

In [14]:
df['genre'].sort_values().unique() # Просмотр уникальных названий жанров

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

Мы просмотрели список и нашли неявные дубликаты жанра `hiphop`:

* *hip*,
* *hop*,
* *hip-hop*.

Чтобы очистить от них таблицу, напишем функцию `replace_wrong_genres()` с двумя параметрами: 
* `wrong_genres` — список дубликатов,
* `correct_genre` — строка с правильным значением.

Функция должна исправить колонку `genre` в таблице `df`: заменить каждое значение из списка `wrong_genres` на значение из `correct_genre`.

In [15]:
def replace_wrong_genres(wrong_genres, correct_genre): # Функция для замены неявных дубликатов
    for wrong_genre in wrong_genres:
        df['genre'] = df['genre'].replace(wrong_genre, correct_genre)

Вызовите `replace_wrong_genres()` и передайте ей такие аргументы, чтобы она устранила неявные дубликаты: вместо `hip`, `hop` и `hip-hop` в таблице должно быть значение `hiphop`:

In [16]:
replace_wrong_genres(['hip', 'hop', 'hip-hop'], 'hiphop') # Устранение неявных дубликатов
replace_wrong_genres(['электроника'], 'electronic')
replace_wrong_genres(['afrikaans'], 'africa')

Проверим, что заменили неправильные названия:

*   hip
*   hop
*   hip-hop

Выводим отсортированный список уникальных значений столбца `genre`:

In [17]:
df['genre'].sort_values().unique() # Проверка на неявные дубликаты

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'alternative',
       'alternativepunk', 'ambient', 'americana', 'animated', 'anime',
       'arabesk', 'arabic', 'arena', 'argentinetango', 'art', 'audiobook',
       'author', 'avantgarde', 'axé', 'baile', 'balkan', 'beats',
       'bigroom', 'black', 'bluegrass', 'blues', 'bollywood', 'bossa',
       'brazilian', 'breakbeat', 'breaks', 'broadway', 'cantautori',
       'cantopop', 'canzone', 'caribbean', 'caucasian', 'celtic',
       'chamber', 'chanson', 'children', 'chill', 'chinese', 'choral',
       'christian', 'christmas', 'classical', 'classicmetal', 'club',
       'colombian', 'comedy', 'conjazz', 'contemporary', 'country',
       'cuban', 'dance', 'dancehall', 'dancepop', 'dark', 'death', 'deep',
       'deutschrock', 'deutschspr', 'dirty', 'disco', 'dnb',
       'documentary', 'downbeat', 'downtempo', 'drum', 'dub', 'dubstep',
       'eastern', 'easy', 'electronic', 'electropop', 'emo', 'entehno',
       'epicmetal', '

Теперь в списке уникальных значений столбца `genre` есть только один вариант жанра хип-хоп.

**Выводы**

Предобработка обнаружила три проблемы в данных:

- нарушения в стиле заголовков,
- пропущенные значения,
- дубликаты — явные и неявные.

Мы исправили заголовки, чтобы упростить работу с таблицей. Без дубликатов исследование станет более точным.

Пропущенные значения мы заменили на `'unknown'`. Ещё предстоит увидеть, не повредят ли исследованию пропуски в колонке `genre`.

Теперь можно перейти к проверке гипотез. 

## Проверка гипотез

### Сравнение поведения пользователей двух столиц

Первая гипотеза утверждает, что пользователи по-разному слушают музыку в Москве и Санкт-Петербурге. Проверим это предположение по данным о трёх днях недели — понедельнике, среде и пятнице. Для этого:

* Разделим пользователей Москвы и Санкт-Петербурга
* Сравним, сколько треков послушала каждая группа пользователей в понедельник, среду и пятницу.


Для тренировки сначала выполним каждый из расчётов по отдельности. 

Оценим активность пользователей в каждом городе. Сгруппируем данные по городу и посчитаем прослушивания в каждой группе.



In [18]:
city_grouping = df.groupby('city')['city'].count() # Подсчёт прослушиваний в каждом городе
city_grouping

city
Moscow              42741
Saint-Petersburg    18512
Name: city, dtype: int64

В Москве прослушиваний больше, чем в Петербурге. Из этого не следует, что московские пользователи чаще слушают музыку. Просто самих пользователей в Москве больше.

Теперь сгруппируем данные по дню недели и подсчитаем прослушивания в понедельник, среду и пятницу. Мы будем учитывать, что в данных есть информация о прослушиваниях только за эти дни.


In [19]:
day_grouping = df.groupby('day')['day'].count() # Подсчёт прослушиваний в каждый из трёх дней
day_grouping

day
Friday       21840
Monday       21354
Wednesday    18059
Name: day, dtype: int64

В среднем пользователи из двух городов менее активны по средам. Но картина может измениться, если рассмотреть каждый город в отдельности.

Мы видели, как работает группировка по городу и по дням недели. Теперь напишем функцию, которая объединит два эти расчёта.

Создадим функцию `number_tracks()`, которая посчитает прослушивания для заданного дня и города. Ей понадобятся два параметра:
* день недели,
* название города.

В функции сохраним в переменную строки исходной таблицы, у которых значение:
  * в колонке `day` равно параметру `day`,
  * в колонке `city` равно параметру `city`.

Для этого применим последовательную фильтрацию с логической индексацией.

Затем посчитаем значения в столбце `user_id` получившейся таблицы. Результат сохраним в новую переменную. Вернём эту переменную из функции.

In [20]:
"""
<создание функции number_tracks()>
Объявляется функция с двумя параметрами: day, city.
В переменной track_list сохраняются те строки таблицы df, для которых значение в столбце 'day' равно параметру day и 
одновременно значение в столбце 'city' равно параметру city (используем последовательную фильтрацию с помощью 
логической индексации).
В переменной track_list_count сохраняется число значений столбца 'user_id', рассчитанное методом count() для таблицы track_list.
Функция возвращает число - значение track_list_count.
Функция для подсчёта прослушиваний для конкретного города и дня.
С помощью последовательной фильтрации с логической индексацией она сначала получит из исходной таблицы строки с нужным днём,
затем из результата отфильтрует строки с нужным городом, методом count() посчитает количество значений в колонке user_id. 
Это количество функция вернёт в качестве результата
"""
def number_tracks(day, city):
    track_list = df[df['day'] == day]
    track_list = track_list[track_list['city'] == city]
    track_list_count = track_list['user_id'].count()
    return track_list_count

Вызовите `number_tracks()` шесть раз, меняя значение параметров — так, чтобы получить данные для каждого города в каждый из трёх дней.

In [21]:
number_tracks('Monday', 'Moscow') # количество прослушиваний в Москве по понедельникам

15740

In [22]:
number_tracks('Monday', 'Saint-Petersburg') # количество прослушиваний в Санкт-Петербурге по понедельникам

5614

In [23]:
number_tracks('Wednesday', 'Moscow') # количество прослушиваний в Москве по средам

11056

In [24]:
number_tracks('Wednesday', 'Saint-Petersburg') # количество прослушиваний в Санкт-Петербурге по средам

7003

In [25]:
number_tracks('Friday', 'Moscow') # количество прослушиваний в Москве по пятницам

15945

In [26]:
number_tracks('Friday', 'Saint-Petersburg') # количество прослушиваний в Санкт-Петербурге по пятницам

5895

Создайте c помощью конструктора `pd.DataFrame` таблицу, где
* названия колонок — `['city', 'monday', 'wednesday', 'friday']`;
* данные — результаты, которые вы получили с помощью `number_tracks`.

In [27]:
# Таблица с результатами
data = [['Moscow', 15740, 11056, 15945],
       ['Saint-Petersburg', 5614, 7003, 5895]]
columns = ['city', 'monday', 'wednesday', 'friday']
pd.DataFrame(data=data, columns=columns)

,city,monday,wednesday,friday
0,Moscow,15740,11056,15945
1,Saint-Petersburg,5614,7003,5895


**Выводы**

Данные показывают разницу поведения пользователей:

- В Москве пик прослушиваний приходится на понедельник и пятницу, а в среду заметен спад.
- В Петербурге, наоборот, больше слушают музыку по средам. Активность в понедельник и пятницу здесь почти в равной мере уступает среде.

Значит, данные говорят в пользу первой гипотезы.

### Музыка в начале и в конце недели

Согласно второй гипотезе, утром в понедельник в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города.

Сохраним таблицы с данными в две переменные:
* по Москве — в `moscow_general`;
* по Санкт-Петербургу — в `spb_general`.

In [28]:
# получение таблицы moscow_general из тех строк таблицы df, 
# для которых значение в столбце 'city' равно 'Moscow'
moscow_general = df[df['city'] == 'Moscow']

In [29]:
# получение таблицы spb_general из тех строк таблицы df,
# для которых значение в столбце 'city' равно 'Saint-Petersburg'
spb_general = df[df['city'] == 'Saint-Petersburg']

Создаём функцию `genre_weekday()` с четырьмя параметрами:
* таблица (датафрейм) с данными,
* день недели,
* начальная временная метка в формате 'hh:mm', 
* последняя временная метка в формате 'hh:mm'.

Функция должна вернуть информацию о топ-10 жанров тех треков, которые прослушивали в указанный день, в промежутке между двумя отметками времени.

In [30]:
"""
Объявление функции genre_weekday() с параметрами table, day, time1, time2, которая возвращает информацию о
самых популярных жанрах в указанный день в заданное время:
1) в переменную genre_df сохраним те строки переданного датафрейма table, для которых одновременно:
    - значение в столбце day равно значению аргумента day
    - значение в столбце time больше значения аргумента time1
    - значение в столбце time меньше значения аргумента time2
Используем последовательную фильтрацию с помощью логической индексации.
2) сгруппируем датафрейм genre_df по столбцу genre, возьмём один из его столбцов и посчитаем методом count()
количество записей для каждого из присутствующих жанров, получившийся Series запишем в переменную genre_df_count
3) отсортируем genre_df_count по убыванию встречаемости и сохраним в переменную genre_df_sorted
4) вернём Series из 10 первых значений genre_df_sorted, это будут топ-10 популярных жанров (в указанный день, в заданное время)
"""
def genre_weekday(table, day, time1, time2):
    genre_df = table[table['day'] == day]
    genre_df = genre_df[genre_df['time'] > time1]
    genre_df = genre_df[genre_df['time'] < time2]
    genre_df_count = genre_df.groupby('genre')['genre'].count()
    genre_df_sorted = genre_df_count.sort_values(ascending=False)
    return genre_df_sorted.head(10)

Cравним результаты функции `genre_weekday()` для Москвы и Санкт-Петербурга в понедельник утром (с 7:00 до 11:00) и в пятницу вечером (с 17:00 до 23:00):

In [31]:
# вызов функции для утра понедельника в Москве
genre_weekday(moscow_general, 'Monday', '07:00', '11:00')

genre
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
unknown        161
Name: genre, dtype: int64

In [32]:
# вызов функции для утра понедельника в Петербурге
genre_weekday(spb_general, 'Monday', '07:00', '11:00')

genre
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
Name: genre, dtype: int64

In [33]:
# вызов функции для вечера пятницы в Москве
genre_weekday(moscow_general, 'Friday', '17:00', '23:00')

genre
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
alternative    163
classical      163
rusrap         142
Name: genre, dtype: int64

In [34]:
# вызов функции для вечера пятницы в Петербурге
genre_weekday(spb_general, 'Friday', '17:00', '23:00')

genre
pop            256
electronic     216
rock           216
dance          210
hiphop          97
alternative     63
jazz            61
classical       60
rusrap          59
world           54
Name: genre, dtype: int64

**Выводы**

Если сравнить топ-10 жанров в понедельник утром, можно сделать такие выводы:

1. В Москве и Петербурге слушают похожую музыку. Единственное отличие — в московский рейтинг вошёл жанр “world”, а в петербургский — джаз и классика.

2. В Москве пропущенных значений оказалось так много, что значение `'unknown'` заняло десятое место среди самых популярных жанров. Значит, пропущенные значения занимают существенную долю в данных и угрожают достоверности исследования.

Вечер пятницы не меняет эту картину. Некоторые жанры поднимаются немного выше, другие спускаются, но в целом топ-10 остаётся тем же самым.

Таким образом, вторая гипотеза подтвердилась лишь частично:
* Пользователи слушают похожую музыку в начале недели и в конце.
* Разница между Москвой и Петербургом не слишком выражена. В Москве чаще слушают русскую популярную музыку, в Петербурге — джаз.

Однако пропуски в данных ставят под сомнение этот результат. В Москве их так много, что рейтинг топ-10 мог бы выглядеть иначе, если бы не утерянные  данные о жанрах.

### Жанровые предпочтения в Москве и Петербурге

Гипотеза: Петербург — столица рэпа, музыку этого жанра там слушают чаще, чем в Москве.  А Москва — город контрастов, в котором, тем не менее, преобладает поп-музыка.

Сгруппируем таблицу `moscow_general` по жанру и посчитаем прослушивания треков каждого жанра методом `count()`. Затем отсортируем результат в порядке убывания и сохраним его в таблице `moscow_genres`.

In [35]:
# группировка таблицы moscow_general по столбцу 'genre', 
# подсчёт числа значений 'genre' в этой группировке методом count(), 
# сортировка получившегося Series в порядке убывания и сохранение в moscow_genres:
moscow_genres = moscow_general.groupby('genre')['genre'].count().sort_values(ascending=False)

Выведем на экран первые десять строк `moscow_genres`:

In [36]:
moscow_genres.head(10) # просмотр первых 10 строк moscow_genres

genre
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: genre, dtype: int64

Теперь повторим то же и для Петербурга.

Сгруппируем таблицу `spb_general` по жанру. Посчитаем прослушивания треков каждого жанра. Результат отсортируем в порядке убывания и сохраним в таблице `spb_genres`:


In [37]:
# группировка таблицы spb_general по столбцу 'genre', 
# подсчёт числа значений 'genre' в этой группировке методом count(), 
# сортировка получившегося Series в порядке убывания и сохранение в spb_genres
spb_genres = spb_general.groupby('genre')['genre'].count().sort_values(ascending=False)

Выведите на экран первые десять строк `spb_genres`:

In [38]:
# просмотр первых 10 строк spb_genres:
spb_genres.head(10)

genre
pop            2431
dance          1932
rock           1879
electronic     1737
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: genre, dtype: int64

**Выводы**

Гипотеза частично подтвердилась:
* Поп-музыка — самый популярный жанр в Москве, как и предполагала гипотеза. Более того, в топ-10 жанров встречается близкий жанр — русская популярная музыка.
* Вопреки ожиданиям, рэп одинаково популярен в Москве и Петербурге. 


## Итоги исследования

Мы проверили три гипотезы и установили:

1. День недели по-разному влияет на активность пользователей в Москве и Петербурге. 

Первая гипотеза полностью подтвердилась.

2. Музыкальные предпочтения не сильно меняются в течение недели — будь то Москва или Петербург. Небольшие различия заметны в начале недели, по понедельникам:
* в Москве слушают музыку жанра “world”,
* в Петербурге — джаз и классику.

Таким образом, вторая гипотеза подтвердилась лишь отчасти. Этот результат мог оказаться иным, если бы не пропуски в данных.

3. Во вкусах пользователей Москвы и Петербурга больше общего чем различий. Вопреки ожиданиям, предпочтения жанров в Петербурге напоминают московские.

Третья гипотеза не подтвердилась. Если различия в предпочтениях и существуют, на основной массе пользователей они незаметны.

**На практике исследования содержат проверки статистических гипотез.**
Из данных одного сервиса не всегда можно сделать вывод о всех жителях города.
Проверки статистических гипотез покажут, насколько они достоверны, исходя из имеющихся данных. 

# Проекты для Яндекс.Практикума
___
Данные проекты были выполнены в ходе обучения в Яндекс.Практикуме по профессии "Аналитик данных" / "Специалист по Data Science".

| Название проекта | Сферы деятельности | Задачи проекта | Ключевые слова | Ключевые навыки |
| :--------------- | :----------------- | :------------- | :------------- | :-------------- |
| Cравнение пользователей сервиса “Яндекс.Музыка” в Москве и Питере | Интернет-сервис, стриминговые сервисы | С помощью библиотеки Pandas и её возможностей проверить данные Яндекс.Музыки и сравнить поведение и предпочтения пользователей Москвы и Санкт-Петербурга. | обработка данных, дубликаты, пропуски, логическая индексация, группировка, сортировка | Python, Pandas |
| Анализ банковских данных (исследование надёжности заёмщиков) | Банковская сфера, кредитование | Исследовать, какие факторы влияют на возврат кредита в срок (например, семейное положение или кол-во детей) | обработка данных, дубликаты, пропуски, категоризация, декомпозиция | предобработка данных, Python, Pandas |
| Успешность компьютерных игр | Gamedev, Интернет-магазины | Выявить закономерности, определяющие успешность компьютерной игры, используя исторические данные о продажах игр, оценки пользователей и экспертов | обработка данных, histogram, boxplot, статистический тест, критерий Стьюдента, piechart | Python, Pandas, NumPy, Matplotlib, предобработка данных, исследовательский анализ данных, описательная статистика, проверка статистических гипотез |
| Анализ убытков приложения | Интернет-сервисы, Стартапы | Разобраться в причинах убытков развлекательного приложения, несмотря на огромные вложения в рекламу, и помочь компании выйти в плюс. | обработка данных, статистический тест, LTV, CAC, когортный анализ | Python, Pandas, Matplotlib, когортный анализ, юнит-экономика, продуктовые метрики, Seaborn |
| A/B-тест. Проверка гипотез | Интернет-магазины | Приоритезировать гипотезы, используя данные интернет-магазина, произвести оценку результатов A/B-тестирования. | A/B-тест, статистический тест, фреймворк, RICE, ICE | Python, Pandas, Matplotlib, SciPy, A/B-тестирование, проверка статистических гипотез |
| Исследования рынка общепита в Москве | Стартапы, Бизнес, Оффлайн | Исследовать рынок общественного питания на основе открытых данных, подготовка презентации для инвесторов | обработка данных, визуализация данных, создание презентаций | Python, Pandas, Seaborn, Plotly, визуализация данных |
| Анализ пользовательского поведения в мобильном приложении | Стартапы, Бизнес, Интернет-сервисы | Проанализировать воронку продаж на основе данных об использовании мобильного предложения, оценить результаты A/A/B-тестирования  | A/B-тест, визуализация, статистический тест | Python, Pandas, Seaborn, Plotly, Matplotlib, A/B-тестирование, событийная аналитика, продуктовые метрики, проверка статистических гипотез, визуализация данных |
| Создание дашборда | Площадки объявлений, Интернет-сервисы | Используя данные канала распространения новостей, статей с сайта построить дашборд с метриками взаимодействия пользователей с карточками статей  | дашборд, пайплайн, Yandex.Cloud, удаленный сервер, виртуальная машина, cron | Python, SQLAlchemy, PostgreSQL, dash, Tableau, продуктовые метрики, построение дашбордов |

Пример описания проекта в README.md - Музыка больших городов
Данные
В наличии были следующие данные о прослушиваниях треков в сервисе Яндекс.Музыка:

Название трека
Исполнитель
Жанр
Город
День прослушивания
Время прослушивания
Задача
Провести сравнение музыкальных предпочтений у пользователей Яндекс.Музыки, проживающих в Москве и Санкт-Петербурге. Сравнить прослушивания в понедельник, среду и пятницу по городам, а также сравнить жанры, которые слушают пользователи утром в понедельник и вечером в пятницу в Москве и Санкт-Петербурге.

Используемые библиотеки
pandas

| Анализ банковских данных (исследование надёжности заёмщиков) | Банковская сфера, кредитование | Исследовать, какие факторы влияют на возврат кредита в срок (например, семейное положение или кол-во детей)На основе статистики о платёжеспособности клиентов 